In [1]:
import re
import pandas as pd
from tqdm import tqdm

# 1. Finding the Unique Words

In [3]:
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())
        
print(len(words))
vocab = list(set(words))
print(len(vocab))

1115585
32198


# 2. Finding the Probability Distribution

In [4]:
word_probability = {}

for word in tqdm(vocab):
    word_probability[word] = float(words.count(word)/len(words))

100%|██████████| 32198/32198 [18:37<00:00, 28.82it/s]


# 3. Text Preprocessing

# Splitting

In [6]:
def split(word):  
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

In [30]:
split('venu')

[('', 'venu'), ('v', 'enu'), ('ve', 'nu'), ('ven', 'u'), ('venu', '')]

# 3.1) Delete

'loave' -> 'love'

In [18]:
def delete(word):
    
    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

# 3.2) Swap

'lvoe' -> 'love'

In [10]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('lvoe')

['vloe', 'love', 'lveo']

# 3.3) Replace

'lave' -> 'love'

In [32]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

print(replace('lave'))
len(replace('lave'))

['aave', 'bave', 'cave', 'dave', 'eave', 'fave', 'gave', 'have', 'iave', 'jave', 'kave', 'lave', 'mave', 'nave', 'oave', 'pave', 'qave', 'rave', 'save', 'tave', 'uave', 'vave', 'wave', 'xave', 'yave', 'zave', 'lave', 'lbve', 'lcve', 'ldve', 'leve', 'lfve', 'lgve', 'lhve', 'live', 'ljve', 'lkve', 'llve', 'lmve', 'lnve', 'love', 'lpve', 'lqve', 'lrve', 'lsve', 'ltve', 'luve', 'lvve', 'lwve', 'lxve', 'lyve', 'lzve', 'laae', 'labe', 'lace', 'lade', 'laee', 'lafe', 'lage', 'lahe', 'laie', 'laje', 'lake', 'lale', 'lame', 'lane', 'laoe', 'lape', 'laqe', 'lare', 'lase', 'late', 'laue', 'lave', 'lawe', 'laxe', 'laye', 'laze', 'lava', 'lavb', 'lavc', 'lavd', 'lave', 'lavf', 'lavg', 'lavh', 'lavi', 'lavj', 'lavk', 'lavl', 'lavm', 'lavn', 'lavo', 'lavp', 'lavq', 'lavr', 'lavs', 'lavt', 'lavu', 'lavv', 'lavw', 'lavx', 'lavy', 'lavz', 'lavea', 'laveb', 'lavec', 'laved', 'lavee', 'lavef', 'laveg', 'laveh', 'lavei', 'lavej', 'lavek', 'lavel', 'lavem', 'laven', 'laveo', 'lavep', 'laveq', 'laver', 'lave

130

# 3.4) Insert

'lve' -> 'love'

In [34]:
def insert(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

print(insert('lve'))
len(insert('lve'))

['alve', 'blve', 'clve', 'dlve', 'elve', 'flve', 'glve', 'hlve', 'ilve', 'jlve', 'klve', 'llve', 'mlve', 'nlve', 'olve', 'plve', 'qlve', 'rlve', 'slve', 'tlve', 'ulve', 'vlve', 'wlve', 'xlve', 'ylve', 'zlve', 'lave', 'lbve', 'lcve', 'ldve', 'leve', 'lfve', 'lgve', 'lhve', 'live', 'ljve', 'lkve', 'llve', 'lmve', 'lnve', 'love', 'lpve', 'lqve', 'lrve', 'lsve', 'ltve', 'luve', 'lvve', 'lwve', 'lxve', 'lyve', 'lzve', 'lvae', 'lvbe', 'lvce', 'lvde', 'lvee', 'lvfe', 'lvge', 'lvhe', 'lvie', 'lvje', 'lvke', 'lvle', 'lvme', 'lvne', 'lvoe', 'lvpe', 'lvqe', 'lvre', 'lvse', 'lvte', 'lvue', 'lvve', 'lvwe', 'lvxe', 'lvye', 'lvze', 'lvea', 'lveb', 'lvec', 'lved', 'lvee', 'lvef', 'lveg', 'lveh', 'lvei', 'lvej', 'lvek', 'lvel', 'lvem', 'lven', 'lveo', 'lvep', 'lveq', 'lver', 'lves', 'lvet', 'lveu', 'lvev', 'lvew', 'lvex', 'lvey', 'lvez']


104

# 4. Finding the Prediction (Level - 1)

# 4.1) Combining Possible Words

In [13]:
def edit(word):   
    return list(set(insert(word) + delete(word) + swap(word) + replace(word)))

# 4.2) Predicting the Word

In [14]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
            
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)


In [24]:
spell_check_edit_1('famili')

['family']

In [25]:
spell_check_edit_1('fameli')

[]

# 5. Finding the Prediction (Level - 2)

# 5.1) Combining Possible Words

In [16]:
def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)       # Level one Edit
    
    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit 
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        
spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'amelie', 'camel']

In [29]:
spell_check_edit_2('ven')

['in', 'he', 'on', 'be', 'her']